In [1]:
import numpy as np
import tensorflow as tf
from lc import analysis, graph, config, loader
from tensorflow.contrib.layers import fully_connected

/home/kaidong/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
np.set_printoptions(precision=2, suppress=True)

In [3]:
def five_layers(x, ref_y):
    activation = graph.max_out
    for i in range(6):
        x = fully_connected(
            x,
            1000,
            activation_fn=activation,
            scope="layer" + str(i))
    y = fully_connected(
        x, 1, activation_fn=tf.identity, scope="fc")
    diff = tf.reduce_sum(y) * -1
    tf.add_to_collection("losses", diff)
    tf.add_to_collection("ptime", y)

    
def gradClips(shape, minEdge, maxEdge, length):
    x = tf.Variable(
        tf.add(tf.multiply(
            tf.random_uniform((length, shape)), (maxEdge-minEdge)),
            minEdge), expected_shape=(length, shape)
    )
    tf.add_to_collection("tVar", x)
    maxVal = np.tile(maxEdge, length).reshape((length, shape))
    minVal = np.tile(minEdge, length).reshape((length, shape))
    
    return tf.clip_by_value(x, minVal, maxVal)

In [4]:
config.DATAFILE = "time_avg"
data = loader.Loader({"name": "GDopt", "discription": ""})

config.NUM_UNIT = 10
minEdge = np.round(np.min(data.X, 0), 1)
maxEdge = np.round(np.max(data.X, 0), 1)

In [5]:
batchSize = 10
gradGraph = tf.Graph()
with gradGraph.as_default():
    five_layers(
        gradClips(31, minEdge, maxEdge, batchSize), np.ones([batchSize])*200
    )
    fin_loss = analysis.fin_loss()
    trainVar, = gradGraph.get_collection("tVar")
    pred, = gradGraph.get_collection("ptime")
    optimizer = tf.train.AdamOptimizer()
    opt = optimizer.minimize(fin_loss, var_list=[trainVar])
    init = tf.global_variables_initializer()
    model = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)[1:]

In [6]:
sess = tf.Session(graph=gradGraph)

with gradGraph.as_default():
    sess.run(init)    
    saver = tf.train.Saver(model)
    saver.restore(sess, tf.train.latest_checkpoint("saved"))
    
gradGraph.finalize()

INFO:tensorflow:Restoring parameters from saved/chkpnt-8000


In [7]:
timelist = []
paralist = []
for i in range(50000):
    _, predVal = sess.run([opt, pred])
    if not (i % 100):
        timelist.append(predVal.reshape(-1))
        paralist.append(sess.run(trainVar))

In [8]:
t = np.array(timelist)

In [9]:
from matplotlib import pyplot as plt
%matplotlib 
plt.style.use('ggplot')

Using matplotlib backend: Qt5Agg


In [10]:
plt.figure()

In [11]:
plt.plot(t)

In [14]:
from xilio import dump

In [15]:
dump("datamax", (paralist, timelist))

In [17]:
optzed = paralist[-1][1]

In [19]:
np.round(optzed, 2)

array([ 15.01,  -4.99,  12.51,  44.98, -15.01,   2.49,  88.01,  58.01,
        28.01, -42.51, -77.51, -42.51, -17.51,  40.99, -27.51,  40.99,
       -57.49,  24.49,  30.01, -22.99, -10.99,  29.49,  70.51,   4.99,
        67.99,  -3.51,  14.49,   0.93,   9.01,  43.51, -15.01], dtype=float32)

In [16]:
timelist

[array([ 85.1 ,  79.16,  85.74,  79.99,  81.24,  74.56,  84.69,  78.72,
         77.31,  69.06], dtype=float32),
 array([ 85.86,  79.3 ,  85.91,  80.14,  81.37,  74.76,  84.86,  78.86,
         77.43,  69.24], dtype=float32),
 array([ 86.62,  79.44,  86.08,  80.3 ,  81.5 ,  74.97,  85.04,  79.  ,
         77.56,  69.42], dtype=float32),
 array([ 87.39,  79.57,  86.25,  80.46,  81.63,  75.17,  85.21,  79.13,
         77.69,  69.6 ], dtype=float32),
 array([ 87.96,  79.71,  86.42,  80.62,  81.76,  75.38,  85.38,  79.27,
         77.83,  69.79], dtype=float32),
 array([ 88.47,  79.84,  86.59,  80.78,  81.9 ,  75.58,  85.56,  79.41,
         77.97,  69.97], dtype=float32),
 array([ 88.98,  79.98,  86.76,  80.94,  82.03,  75.78,  85.73,  79.55,
         78.11,  70.15], dtype=float32),
 array([ 89.46,  80.12,  86.93,  81.1 ,  82.16,  75.89,  85.9 ,  79.68,
         78.24,  70.33], dtype=float32),
 array([ 89.95,  80.25,  87.1 ,  81.26,  82.29,  76.01,  86.08,  79.82,
         78.37,  70.51],

In [13]:
min(data.Y)

array([ 63.69])

In [ ]:
15.01,  -4.99,  12.51,  44.98, -15.01,   2.49,  88.01,  58.01,
        28.01, -42.51, -77.51, -42.51, -17.51,  40.99, -27.51,  40.99,
       -57.49,  24.49,  30.01, -22.99, -10.99,  29.49,  70.51,   4.99,
        67.99,  -3.51,  14.49,   0.93,   9.01,  43.51, -15.01